<a href="https://colab.research.google.com/github/zhgjenny93/NLP-Thinkful/blob/main/Chatbot_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment, you'll work with a dataset called [Cornell Movie--Dialogs Corpus](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html), which was released by the Cornell University. The dataset contains conversations from more than 600 movies. Use the following credentials to access the dataset from the Thinkful database.

```
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'
```
The data is in the table called dialogs.

1. First, do some data preprocessing to clean up the data. You can use your solution to the assignment of the Text preprocessing checkpoint.

2. Develop a chatbot using this corpus. In doing this, you're free to choose a chatbot development library like ChatterBot or write your own code from scratch.

3. Start a conversation with your chatbot, and discuss its strengths and weaknesses.

In [1]:
import nltk
import numpy as np
import pandas as pd
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import spacy
from sqlalchemy import create_engine
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

dialogs_df = pd.read_sql_query('select * from dialogs', con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

dialogs_df.head(10)

,index,dialogs
0,0,Can we make this quick? Roxanne Korrine and A...
1,1,"Well, I thought we'd start with pronunciation,..."
2,2,Not the hacking and gagging and spitting part....
3,3,Okay... then how 'bout we try out some French ...
4,4,You're asking me out. That's so cute. What's ...
5,5,Forget it.
6,6,"No, no, it's my fault -- we didn't have a prop..."
7,7,Cameron.
8,8,"The thing is, Cameron -- I'm at the mercy of a..."
9,9,Seems like she could get a date easy enough...


In [3]:
# Remove new lines and other extra whitespace by creating str from df and splitting and rejoining

dialogs_str = ' '.join(dialogs_df.dialogs)
dialogs_str = ' '.join(dialogs_str.split())

In [4]:
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done here, may take a while
dialogs_doc = nlp(dialogs_str)

In [5]:
dialog_sents = [sent.text for sent in dialogs_doc.sents if len(sent.text) > 1]

In [6]:
GREETING_INPUTS = ['hello', 'hi', 'greetings', "what's up", 'hey', 'yo','heya','hiya']
GREETING_RESPONSES = ['hello', 'hi', 'hey', 'hi there']
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
      return random.choice(GREETING_RESPONSES)

In [7]:
# pip install chatterbot

In [8]:
# Import libraries
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement

In [ ]:
# Create a chatbot
chatbot = ChatBot('Dialogs')

# This is to remove the accumulated knowledge base
chatbot.storage.drop()

# Create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

# Train the chatbot based on dialogs
trainer.train(dialog_sents)

List Trainer: [##                  ] 8%

In [ ]:
print("Cornell: I will try to respond to you reasonably. If you want to exit, type bye")

# Below is the chatting
while True:
  user_input = input("User: ")
  user_input = user_input.lower()

  if(user_input != 'bye'):
    if (user_input == 'thanks' or user_input == 'thank you'):
      print("Cornell: You're welcome.")
      break
    else:
      if (greeting(user_input) != None):
        print("Cornell: " + greeting(user_input))
      else:
        print("Cornell: ", end ="")
        print(chatbot.get_response(user_input))
  else:
    print("Cornell: Bye! It was a great chat.")
    break